In [ ]:
import pandas as pd
import numpy as np
import operator
import time
from sklearn.model_selection import train_test_split
from operator import itemgetter
from collections import OrderedDict
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
UI=pd.read_csv("H:/LinkPrediction/user-user-item network/Epinions/UserItem.csv")

S=pd.read_csv("H:/LinkPrediction/user-user-item network/Epinions/Social.csv")

In [ ]:
S=S.sort_values(by=['userID1'])
S=S.reset_index(drop=True)

In [ ]:
del UI['Index']

In [ ]:
UI_train, UI_test = train_test_split(UI, test_size=0.1)
UI_train=UI_train.sort_values(by='userID')
UI_train=UI_train.reset_index(drop=True)
UI_test=UI_test.sort_values(by='userID')
UI_test=UI_test.reset_index(drop=True)

In [ ]:
all_user_node=UI.userID.unique()
all_item_node=UI.itemID.unique()

In [ ]:
user_node_num=UI_train.userID.unique()
len(user_node_num)
user_node_num
item_node_num=UI_train.itemID.unique()
len(item_node_num)

In [ ]:
user_node_num=np.sort(user_node_num)
item_node_num=np.sort(item_node_num)

In [ ]:
UIM=np.zeros((len(user_node_num),len(item_node_num)))
for i in range(len(UI_train)):
    UIM[UI_train.userID[i]-1,UI_train.itemID[i]-1]=1

In [ ]:
UIM

In [ ]:
len(S.userID1)

In [ ]:
user_node_num

In [ ]:
SM=np.zeros((len(user_node_num),len(user_node_num))) 
for i in range(len(S.userID1)):                  
    SM[S.userID1[i]-1,S.userID2[i]-1]=1 

In [ ]:
SM

In [ ]:
np.shape(SM)

In [ ]:
#One way to generate the matrix for the further recommendation list
P=0.5
ui=0
uu=0
w=np.zeros((len(item_node_num),len(item_node_num)))
for alpha in range(len(item_node_num)):
    for beta in range(len(item_node_num)):
        related_users_num=len(np.where(UIM[:,beta]==1)[0])
        for i in range(len(user_node_num)):
            collected_item_num=len(np.where(UIM[i,:]==1)[0])  
            ui += (UIM[i,alpha]*UIM[i,beta])/collected_item_num  
            for j in range(len(user_node_num)):
                if UIM[i,alpha]==0 or SM[i,j]==0 or UIM[j,beta]==0:
                    continue
                uu += (UIM[i,alpha]*SM[i,j]*UIM[j,beta])/(collected_item_num*len(np.where(SM[j,:]==1)[0]))
        first_half=P*(1/related_users_num)*ui
        ui=0
        second_half=(1-P)*(1/related_users_num)*uu
        uu=0
        w[alpha,beta]=first_half+second_half

In [ ]:
# One way to generate the further recommendation list using matrix approach
import time
start =time.clock()
P=0.5
ui=0
uu=0
w=np.zeros((len(item_node_num),len(item_node_num)))
for alpha in range(len(item_node_num)):
    for beta in range(len(item_node_num)):
        for i in range(len(user_node_num)):
            ui += (test0[i,alpha]*test0[i,beta])/len(np.where(test0[i,:]==1)[0])
            for j in range(len(user_node_num)):
                if test0[i,alpha]==0 or test1[i,j]==0 or test0[j,beta]==0:
                    continue
                uu += (test0[i,alpha]*test1[i,j]*test0[j,beta])/(len(np.where(test0[i,:]==1)[0])*len(np.where(test1[j,:]==1)[0]))
        first_half=P*(1/len(np.where(test0[:,beta]==1)[0]))*ui
        ui=0
        second_half=(1-P)*(1/len(np.where(test0[:,beta]==1)[0]))*uu
        uu=0
        w[alpha,beta]=first_half+second_half
end = time.clock()
print('Running time: %s Seconds'%(end-start))

In [ ]:
# The other way to generate the further recommendation
def aSMD(ID,aP):
    user_ereg_1=dict.fromkeys(user_node_num-1, 0) 
    user_ereg_2=dict.fromkeys(user_node_num-1, 0)
    item_ereg=dict.fromkeys(item_node_num-1, 0)
    uncollect = np.where(UIM[np.where(user_node_num==ID)[0][0]]==0)[0] 
    recommendlist=dict.fromkeys(uncollect, 0)  
    collect = np.where(UIM[np.where(user_node_num==ID)[0][0]]==1)[0]
    #Step1
    for i in collect:
        related_user=np.where(UIM[:,i]==1)
        i2u_edge= 1/len(related_user[0])
        for j in related_user[0]: 
            user_ereg_1[j] += i2u_edge
    #Step2
    for p in range(0,len(user_node_num)):
        user_ereg_2[p]=user_ereg_1[p]*aP 
        user_friends=np.where(SM[p,:]==1)
        for q in user_friends[0]:
            user_ereg_2[p]+=user_ereg_1[q]*(1-aP)/len(np.where(SM[q,:]==1)[0])
    #Step3   
    for m in range(0,len(user_node_num)):
        if(user_ereg_2[m]==0):
            continue
        related_item=np.where(UIM[m,:]==1)
        u2i_edge= user_ereg_2[m]/len(related_item[0]) 
        for n in related_item[0]:
            item_ereg[n]+=u2i_edge
    
    for recommendItem in recommendlist:
        recommendlist[recommendItem]=item_ereg[recommendItem]
    return OrderedDict(sorted(recommendlist.items(), key=itemgetter(1),reverse=True))

In [ ]:
#Evaluation Metrics
start =time.clock()
precision_list=[]
all_RS_list=[]
all_diver=[]
item_similarity=cosine_similarity(UIM.T)
L=20
temp=0
common_users=sorted(list(set(UI_train.userID)&set(UI_test.userID)))
#Precision
aP=[0,0.2,0.4,0.6,0.8,1.0]
for p in aP:
    for i in common_users:
        rec= aSMD(i,p)
        L_rec=list(rec.items())[:L]
        precision=len(list(set(UI_test.itemID[UI_test.userID==i])& set([q for q in rec][:L])))/L
        precision_list.append(precision)

    #Ranking Score
        common_items= UI_test.itemID[UI_test.userID==i]
        loc=[list(rec.keys()).index(w-1) for w in common_items if w-1 in rec.keys()]
        RS= [(j+1)/len(rec) for j in loc]
        item_RS=np.mean(RS)
        all_RS_list.append(item_RS)

    #Intra-user diversity
        for item in range(len(L_rec)):
            for item1 in range(len(L_rec)):
                if  item<=item1:
                    continue
                temp += item_similarity[L_rec[item][0],L_rec[item1][0]]#***这里为什么用到L_rec[item][0]？两item的相似性是否为item_similarity[item1_ID，item2_ID]
        diver=(1/(L*(L-1)))*temp
        all_diver.append(diver)
        temp=0

    RS_SMD=np.mean(all_RS_list)
    precision_SMD = np.mean(precision_list)
    diversity_SMD=np.mean(all_diver)

    print('P is %s'%p)
    print(RS_SMD)
    print(precision_SMD)
    print(diversity_SMD)
    end = time.clock()
    print('Running time: %s Seconds'%(end-start))